In [1]:
import pandas as pd
import seaborn as sns 
import numpy as np
import glob
import matplotlib.pyplot as plt
from scipy.stats import sem
%matplotlib inline
%config Completer.use_jedi = False


#import required sklearn

import sklearn as sklearn
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error, classification_report, confusion_matrix,roc_auc_score, f1_score
from sklearn.utils import resample
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score, StratifiedKFold, GridSearchCV, RepeatedStratifiedKFold, RepeatedKFold, cross_validate
from sklearn.feature_selection import mutual_info_classif,VarianceThreshold, SelectFromModel
from sklearn.svm import LinearSVC, SVC
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression, Lasso
from sklearn.decomposition import PCA

In [2]:
#Load the filtered isoform switch, eQTLs and filtered survival files

isoform_switch_file = '/Users/jake/OneDrive - University of Glasgow/Project/BRCA/isoform_switch/BRCA_FPKM_protein_isoform_filtered_matched.csv'
isoform_switch = pd.read_csv(isoform_switch_file,header=[0,1],sep='\t',index_col=[0])

cis_95_file = '/Users/jake/OneDrive - University of Glasgow/Project/BRCA/eQTL/eQTL_95/BRCA_FPKM_cis_eQTL_95_matched.csv'
trans_95_file = '/Users/jake/OneDrive - University of Glasgow/Project/BRCA/eQTL/eQTL_95/BRCA_FPKM_trans_eQTL_95_matched.csv'

cis_95 = pd.read_csv(cis_95_file,header=[0,1],sep='\t',index_col=[0])
trans_95 = pd.read_csv(trans_95_file,header=[0,1],sep='\t',index_col=[0])

isoform_survival_file = '/Users/jake/OneDrive - University of Glasgow/Project/BRCA/Survival/BRCA_isoform_survival.csv'
cis_survival_file = '/Users/jake/OneDrive - University of Glasgow/Project/BRCA/Survival/BRCA_cis_95_survival.csv'
trans_survival_file = '/Users/jake/OneDrive - University of Glasgow/Project/BRCA/Survival/BRCA_trans_95_survival.csv'

isoform_survival = pd.read_csv(isoform_survival_file,header=[0,1],sep='\t',index_col=[0])
cis_survival = pd.read_csv(cis_survival_file,header=[0,1],sep='\t',index_col=[0])
trans_survival = pd.read_csv(trans_survival_file,header=[0,1],sep='\t',index_col=[0])

In [3]:
isoform_survival

Gene_symbol,TANK,RBCK1,CASC4,DUT,TMEM201,FBLN2,ABLIM3,MPRIP,MCF2L,EHBP1,...,ADAMTS12,PCGF5,REPS1,CCND3,LRRFIP2,ZRANB2,H2AFV,CEP104,SLK,Three_year_status
Gene_ensembl_id,ENSG00000136560,ENSG00000125826,ENSG00000166734,ENSG00000128951,ENSG00000188807,ENSG00000163520,ENSG00000173210,ENSG00000133030,ENSG00000126217,ENSG00000115504,...,ENSG00000151388,ENSG00000180628,ENSG00000135597,ENSG00000112576,ENSG00000093167,ENSG00000132485,ENSG00000105968,ENSG00000116198,ENSG00000065613,Three_year_status
TCGA-BH-A1FN,15.343920,16.896624,16.751647,9.053910,3.469054,18.969059,2.182001,6.682397,1.918617,7.225800,...,7.858756,4.875201,13.634318,9.727431,4.229633,25.029257,31.033869,6.468174,35.239420,1
TCGA-E9-A1NG,8.204818,15.724169,30.831324,9.540387,1.458159,41.826503,7.087709,9.316663,1.614837,7.243868,...,13.075343,7.006416,4.295827,10.860205,6.456335,11.036110,25.771819,5.205502,15.373430,0
TCGA-AR-A5QQ,4.584809,37.413581,7.432917,31.326441,3.784303,26.942907,4.418783,5.318138,0.278765,8.252696,...,3.191489,2.414725,5.561334,21.714776,7.972584,11.563606,32.334497,4.179797,14.306222,0
TCGA-AR-A24L,12.187805,11.817051,42.466447,17.772046,1.528907,29.601257,5.565343,7.070899,1.737580,6.467532,...,18.151597,8.583196,4.694506,10.228295,5.355158,17.109764,40.169837,5.040365,14.045827,1
TCGA-AC-A23H,10.839375,17.725220,17.908471,10.923948,2.476160,19.744676,5.266147,4.993341,1.905335,4.650695,...,8.015156,3.409479,4.394309,8.867406,5.226943,13.281163,34.015799,6.835127,11.712936,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA-AR-A2LH,9.579705,23.338886,20.126391,23.172247,3.533242,32.862378,3.116353,7.454451,1.688323,3.913063,...,1.487765,5.793914,4.752568,24.878755,3.930242,12.494149,32.978206,5.631005,15.788019,0
TCGA-B6-A0IN,5.653001,12.949612,13.825296,9.798449,1.953969,22.035637,4.326962,5.437337,1.373668,3.379278,...,3.018018,3.847809,2.247135,13.729128,2.975186,8.600656,39.169583,2.867297,4.809616,1
TCGA-B6-A0IK,6.527808,11.453830,16.448324,19.178110,3.168995,13.938611,4.866697,4.597985,2.722875,3.955684,...,2.511905,3.526115,4.455779,10.806836,5.182330,12.908030,52.973266,5.113113,5.495684,0


In [4]:
#Check for and missing data

display(isoform_switch.isnull().sum().unique())
display(cis_95.isnull().sum().unique())
display(trans_95.isnull().sum().unique())
display(isoform_survival.isnull().sum().unique())
display(cis_survival.isnull().sum().unique())
display(trans_survival.isnull().sum().unique())

array([0])

array([0])

array([0])

array([0])

array([0])

array([0])

In [5]:
#Split data into features (X) and target (y) ... no need to to train_test_split since gridsearch and cv accounts for split

X_iso = isoform_switch.drop('Cancer',axis=1,level=0)
y_iso = isoform_switch['Cancer','Cancer']

X_cis_95 = cis_95.drop('Cancer',axis=1,level=0)
y_cis_95 = cis_95['Cancer','Cancer']

X_trans_95 = trans_95.drop('Cancer',axis=1,level=0)
y_trans_95 = trans_95['Cancer','Cancer']

X_iso_sur = isoform_survival.drop('Three_year_status',axis=1,level=0)
y_iso_sur = isoform_survival['Three_year_status','Three_year_status']

X_cis_sur = cis_survival.drop('Three_year_status',axis=1,level=0)
y_cis_sur = cis_survival['Three_year_status','Three_year_status']

X_trans_sur = trans_survival.drop('Three_year_status',axis=1,level=0)
y_trans_sur = trans_survival['Three_year_status','Three_year_status']

In [6]:
def compare_models(X,y,data_name):
    '''Function to compare models. Takes in X and y, alongside data_name string for printing'''
    
    #Create list of different classifiers and feature_selection to try

    clfs = [LogisticRegression(),SVC(),RandomForestClassifier()]

    #Create list for corresponding feature selection
    
    feat_sels = [LogisticRegression() ,LinearSVC(), RandomForestClassifier()]
    
    #repeated stratified 3 fold
    r_stratkfold = RepeatedStratifiedKFold(n_splits=3, n_repeats=3)

    #Create pipeline with abertrary classifier and feature_selection
    pipeline = Pipeline([
        ('scale', StandardScaler()), #Step1 - normalize data
        ('pca',PCA(n_components=5)), #Step 2 - PCA Dimensionality reduction
        ('feature_selection', SelectFromModel(LogisticRegression())), #Step 3 - feature selection (LR used as default)
        ('classifier', LogisticRegression()) #Step 4 - classifier (LR used as default)
    ])

    
    #for loop to loop over each classifier and its corresponding feature selection classifier
    #Performs CV and returns auroc score (alongside other metrics)
    
    for index, clf in enumerate(clfs):
        pipeline.set_params(classifier = clf,feature_selection = SelectFromModel(feat_sels[index]))
        scores = cross_validate(pipeline, X, y,scoring=('f1_weighted','roc_auc'),cv=r_stratkfold)
        print('---------------------------------')
        print(str(clf),'-',str(data_name))
        print('-----------------------------------')
        for key, values in scores.items():
                print(key,' mean ', round(values.mean(),2))
                print(key,' SD ', round(np.std(values),2))

In [ ]:
#Run the above function on each set of data

In [7]:
#Isoform switch tumour or normal

compare_models(X_iso,y_iso,data_name='isoform switches')

---------------------------------
LogisticRegression() - isoform switches
-----------------------------------
fit_time  mean  0.02
fit_time  SD  0.01
score_time  mean  0.01
score_time  SD  0.0
test_f1_weighted  mean  0.96
test_f1_weighted  SD  0.01
test_roc_auc  mean  0.99
test_roc_auc  SD  0.0
---------------------------------
SVC() - isoform switches
-----------------------------------
fit_time  mean  0.01
fit_time  SD  0.0
score_time  mean  0.01
score_time  SD  0.0
test_f1_weighted  mean  0.94
test_f1_weighted  SD  0.02
test_roc_auc  mean  0.99
test_roc_auc  SD  0.0
---------------------------------
RandomForestClassifier() - isoform switches
-----------------------------------
fit_time  mean  0.39
fit_time  SD  0.08
score_time  mean  0.07
score_time  SD  0.0
test_f1_weighted  mean  0.92
test_f1_weighted  SD  0.03
test_roc_auc  mean  0.98
test_roc_auc  SD  0.02


In [8]:
#Cis-eQTL tumour or normal

compare_models(X_cis_95,y_cis_95,'cis eQTL')

---------------------------------
LogisticRegression() - cis eQTL
-----------------------------------
fit_time  mean  0.04
fit_time  SD  0.01
score_time  mean  0.02
score_time  SD  0.0
test_f1_weighted  mean  0.97
test_f1_weighted  SD  0.02
test_roc_auc  mean  0.98
test_roc_auc  SD  0.01


/Users/jake/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/jake/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/jake/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/jake/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/jake/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:986: ConvergenceWarning: Liblinear failed to converge, increa

---------------------------------
SVC() - cis eQTL
-----------------------------------
fit_time  mean  0.04
fit_time  SD  0.03
score_time  mean  0.03
score_time  SD  0.01
test_f1_weighted  mean  0.97
test_f1_weighted  SD  0.02
test_roc_auc  mean  0.99
test_roc_auc  SD  0.01
---------------------------------
RandomForestClassifier() - cis eQTL
-----------------------------------
fit_time  mean  0.43
fit_time  SD  0.06
score_time  mean  0.09
score_time  SD  0.01
test_f1_weighted  mean  0.9
test_f1_weighted  SD  0.04
test_roc_auc  mean  0.96
test_roc_auc  SD  0.02


In [9]:
#Trans eQTL tumour normal

compare_models(X_trans_95,y_trans_95,'trans eQTL')

---------------------------------
LogisticRegression() - trans eQTL
-----------------------------------
fit_time  mean  0.03
fit_time  SD  0.01
score_time  mean  0.01
score_time  SD  0.0
test_f1_weighted  mean  0.96
test_f1_weighted  SD  0.01
test_roc_auc  mean  0.99
test_roc_auc  SD  0.01


/Users/jake/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/jake/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/jake/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/jake/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/jake/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:986: ConvergenceWarning: Liblinear failed to converge, increa

---------------------------------
SVC() - trans eQTL
-----------------------------------
fit_time  mean  0.03
fit_time  SD  0.02
score_time  mean  0.02
score_time  SD  0.01
test_f1_weighted  mean  0.96
test_f1_weighted  SD  0.02
test_roc_auc  mean  0.99
test_roc_auc  SD  0.01
---------------------------------
RandomForestClassifier() - trans eQTL
-----------------------------------
fit_time  mean  0.45
fit_time  SD  0.08
score_time  mean  0.09
score_time  SD  0.01
test_f1_weighted  mean  0.93
test_f1_weighted  SD  0.06
test_roc_auc  mean  0.97
test_roc_auc  SD  0.04


In [10]:
#isoform switch survival

compare_models(X_iso_sur,y_iso_sur,'isoform survival')

---------------------------------
LogisticRegression() - isoform survival
-----------------------------------
fit_time  mean  0.02
fit_time  SD  0.01
score_time  mean  0.01
score_time  SD  0.0
test_f1_weighted  mean  0.55
test_f1_weighted  SD  0.04
test_roc_auc  mean  0.55
test_roc_auc  SD  0.07


/Users/jake/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/jake/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/jake/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/jake/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/jake/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:986: ConvergenceWarning: Liblinear failed to converge, increa

---------------------------------
SVC() - isoform survival
-----------------------------------
fit_time  mean  0.01
fit_time  SD  0.0
score_time  mean  0.01
score_time  SD  0.0
test_f1_weighted  mean  0.53
test_f1_weighted  SD  0.05
test_roc_auc  mean  0.57
test_roc_auc  SD  0.07
---------------------------------
RandomForestClassifier() - isoform survival
-----------------------------------
fit_time  mean  0.38
fit_time  SD  0.06
score_time  mean  0.07
score_time  SD  0.0
test_f1_weighted  mean  0.56
test_f1_weighted  SD  0.07
test_roc_auc  mean  0.57
test_roc_auc  SD  0.07


In [11]:
#cis-eQTL survival

compare_models(X_cis_sur,y_cis_sur,'cis eQTL survival')

---------------------------------
LogisticRegression() - cis eQTL survival
-----------------------------------
fit_time  mean  0.03
fit_time  SD  0.01
score_time  mean  0.02
score_time  SD  0.0
test_f1_weighted  mean  0.56
test_f1_weighted  SD  0.06
test_roc_auc  mean  0.6
test_roc_auc  SD  0.06


/Users/jake/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/jake/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/jake/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/jake/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/jake/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:986: ConvergenceWarning: Liblinear failed to converge, increa

---------------------------------
SVC() - cis eQTL survival
-----------------------------------
fit_time  mean  0.04
fit_time  SD  0.02
score_time  mean  0.03
score_time  SD  0.02
test_f1_weighted  mean  0.53
test_f1_weighted  SD  0.06
test_roc_auc  mean  0.59
test_roc_auc  SD  0.06
---------------------------------
RandomForestClassifier() - cis eQTL survival
-----------------------------------
fit_time  mean  0.41
fit_time  SD  0.07
score_time  mean  0.09
score_time  SD  0.0
test_f1_weighted  mean  0.55
test_f1_weighted  SD  0.06
test_roc_auc  mean  0.57
test_roc_auc  SD  0.05


In [12]:
#Trans-eQTL survival

compare_models(X_trans_sur,y_trans_sur,'trans eQTL survival')

---------------------------------
LogisticRegression() - trans eQTL survival
-----------------------------------
fit_time  mean  0.04
fit_time  SD  0.01
score_time  mean  0.02
score_time  SD  0.01
test_f1_weighted  mean  0.47
test_f1_weighted  SD  0.05
test_roc_auc  mean  0.45
test_roc_auc  SD  0.07


/Users/jake/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/jake/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/jake/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/jake/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/jake/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:986: ConvergenceWarning: Liblinear failed to converge, increa

---------------------------------
SVC() - trans eQTL survival
-----------------------------------
fit_time  mean  0.03
fit_time  SD  0.01
score_time  mean  0.03
score_time  SD  0.01
test_f1_weighted  mean  0.46
test_f1_weighted  SD  0.06
test_roc_auc  mean  0.5
test_roc_auc  SD  0.04
---------------------------------
RandomForestClassifier() - trans eQTL survival
-----------------------------------
fit_time  mean  0.46
fit_time  SD  0.12
score_time  mean  0.1
score_time  SD  0.02
test_f1_weighted  mean  0.46
test_f1_weighted  SD  0.07
test_roc_auc  mean  0.45
test_roc_auc  SD  0.08
